In [3]:
import pandas as pd

pd.options.display.max_colwidth = 500

compute_samples = pd.read_csv('../sourmash/lung_cancer_v4/compute/samples.csv')
print(compute_samples.shape)
compute_samples.head()

(5054, 10)


,id,read1,read2,name,output,trim_low_abundance_kmers,dna,protein,ksizes,scaled
0,A10_B000419_S34,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000419_S34/A10_B000419_S34_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000419_S34/A10_B000419_S34_R2_001.fastq.gz,A10_B000419_S34,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,True,True,True,"21,27,33,51",1000
1,A10_B000420_S82,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000420_S82/A10_B000420_S82_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B000420_S82/A10_B000420_S82_R2_001.fastq.gz,A10_B000420_S82,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,True,True,True,"21,27,33,51",1000
2,A10_B002073_S166,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002073_S166/A10_B002073_S166_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002073_S166/A10_B002073_S166_R2_001.fastq.gz,A10_B002073_S166,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002073_S166.signature,True,True,True,"21,27,33,51",1000
3,A10_B002078_S202,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002078_S202/A10_B002078_S202_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002078_S202/A10_B002078_S202_R2_001.fastq.gz,A10_B002078_S202,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002078_S202.signature,True,True,True,"21,27,33,51",1000
4,A10_B002095_S118,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002095_S118/A10_B002095_S118_R1_001.fastq.gz,s3://czbiohub-seqbot/fastqs/180516_A00111_0149_AH5CM2DSXX/rawdata/A10_B002095_S118/A10_B002095_S118_R2_001.fastq.gz,A10_B002095_S118,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002095_S118.signature,True,True,True,"21,27,33,51",1000


## look at aws folder and save aws output to file

In [7]:
prefix = 's3://olgabot-maca/facs/sourmash_index_all'
txt = 'sourmash_databases.txt'

! aws s3 ls $prefix/ > $txt
! cat $txt

                           PRE tabula-muris-k21-dna/
                           PRE tabula-muris-k21-protein/
                           PRE tabula-muris-k27-dna/
                           PRE tabula-muris-k27-protein/
                           PRE tabula-muris-k33-dna/
                           PRE tabula-muris-k33-protein/
                           PRE tabula-muris-k51-dna/
                           PRE tabula-muris-k51-protein/


Running each line one-by-one is left as an exercise to the reader :)

In [8]:
databases = pd.read_table(txt, delim_whitespace=True, header=None, names=['is_prefix', 'database_name'])
databases['database_name'] = databases['database_name'].str.strip('/')
databases = databases.drop('is_prefix', axis=1)
databases['ksize'] = databases['database_name'].str.extract('k(\d+)').astype(int)
databases['sequence_to_compare'] = databases['database_name'].map(lambda x: x.split('-')[-1])
databases['database'] = databases['database_name'].map(lambda x: f'{prefix}/{x}/{x}/')
databases = databases.set_index('database_name')
databases

,ksize,sequence_to_compare,database
database_name,,,
tabula-muris-k21-dna,21,dna,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-dna/tabula-muris-k21-dna/
tabula-muris-k21-protein,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
tabula-muris-k27-dna,27,dna,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-dna/tabula-muris-k27-dna/
tabula-muris-k27-protein,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
tabula-muris-k33-dna,33,dna,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-dna/tabula-muris-k33-dna/
tabula-muris-k33-protein,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
tabula-muris-k51-dna,51,dna,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-dna/tabula-muris-k51-dna/
tabula-muris-k51-protein,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/


### Only compare on protein databases

In [9]:
protein_databases = databases.query('sequence_to_compare == "protein"')
protein_databases

,ksize,sequence_to_compare,database
database_name,,,
tabula-muris-k21-protein,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
tabula-muris-k27-protein,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
tabula-muris-k33-protein,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
tabula-muris-k51-protein,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/


## "Multiply" the samples x databases x ignore abundances to get every combination

Use `product` from Python's [itertools](https://docs.python.org/3/library/itertools.html) which is my favorite standard library module.

Remember that Python was designed as "batteries included" so if you're doing something like 

In [10]:
import itertools

ignore_abundances = True, False

data = list(itertools.product(ignore_abundances, compute_samples['output'], 
                              protein_databases.index))

samples = pd.DataFrame(data, columns=['ignore_abundance', 'signature', 'database_name'])
print(samples.shape)
samples

(40432, 3)


,ignore_abundance,signature,database_name
0,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k21-protein
1,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k27-protein
2,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k33-protein
3,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k51-protein
4,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k21-protein
5,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k27-protein
6,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k33-protein
7,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k51-protein
8,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002073_S166.signature,tabula-muris-k21-protein
9,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B002073_S166.signature,tabula-muris-k27-protein


In [11]:
import os

samples['sample_id'] = samples['signature'].map(lambda x: os.path.basename(x).split('.')[0])
samples.head()

,ignore_abundance,signature,database_name,sample_id
0,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k21-protein,A10_B000419_S34
1,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k27-protein,A10_B000419_S34
2,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k33-protein,A10_B000419_S34
3,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k51-protein,A10_B000419_S34
4,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k21-protein,A10_B000420_S82


In [13]:
output_prefix = f's3://olgabot-maca/lung_cancer/sourmash_search/'

samples['output'] = output_prefix + samples['database_name'] + '/' + samples['id'] + '.csv'
samples['output'].head()

0    s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/A10_B000419_S34_ignore-abundance=True.csv
1    s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k27-protein/A10_B000419_S34_ignore-abundance=True.csv
2    s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k33-protein/A10_B000419_S34_ignore-abundance=True.csv
3    s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000419_S34_ignore-abundance=True.csv
4    s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/A10_B000420_S82_ignore-abundance=True.csv
Name: output, dtype: object

In [27]:
samples_databases = samples.join(protein_databases, on='database_name')
print(samples_databases.shape)
samples_databases.head()

(40432, 9)


,ignore_abundance,signature,database_name,sample_id,id,output,ksize,sequence_to_compare,database
0,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k21-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/A10_B000419_S34_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
1,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k27-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k27-protein/A10_B000419_S34_ignore-abundance=True.csv,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
2,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k33-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k33-protein/A10_B000419_S34_ignore-abundance=True.csv,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
3,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k51-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000419_S34_ignore-abundance=True.csv,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/
4,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k21-protein,A10_B000420_S82,A10_B000420_S82_ignore-abundance=True,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/A10_B000420_S82_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/


In [29]:
samples_databases['id'] = samples_databases.apply(lambda x: 
                              '{sample_id}_ignore-abundance={ignore_abundance}_{database_name}'.format(**x), 
                              axis=1)
samples_databases.head()

,ignore_abundance,signature,database_name,sample_id,id,output,ksize,sequence_to_compare,database
0,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k21-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True_tabula-muris-k21-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/A10_B000419_S34_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
1,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k27-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True_tabula-muris-k27-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k27-protein/A10_B000419_S34_ignore-abundance=True.csv,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
2,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k33-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True_tabula-muris-k33-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k33-protein/A10_B000419_S34_ignore-abundance=True.csv,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
3,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000419_S34.signature,tabula-muris-k51-protein,A10_B000419_S34,A10_B000419_S34_ignore-abundance=True_tabula-muris-k51-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/A10_B000419_S34_ignore-abundance=True.csv,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/
4,True,s3://olgabot-maca/lung_cancer/sourmash_v4/A10_B000420_S82.signature,tabula-muris-k21-protein,A10_B000420_S82,A10_B000420_S82_ignore-abundance=True_tabula-muris-k21-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/A10_B000420_S82_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/


In [30]:
chosen_ids = ['C14_B003528_S62',
 'D1_B003125_S25',
 'E19_B003570_S199',
 'F21_B000420_S213',
 'G10_B003586_S142',
 'G4_B003570_S232',
 'G9_B003511_S57',
 'H7_B003588_S211',
 'I22_B002095_S22',
 'I3_B003573_S63',
 'J11_B003573_S95',
 'J8_B003528_S224',
 'K7_B002073_S103',
 'L16_B003588_S16',
 'L5_B003588_S5',
 'M1_B000420_S61',
 'M23_B002097_S251',
 'N15_B000420_S99',
 'O3_B003573_S207',
 'P14_B000420_S146',
 'P2_B003125_S14']

samples_subset = samples_databases.query('sample_id in @chosen_ids')
print(samples_subset.shape)
samples_subset.head()

(168, 9)


,ignore_abundance,signature,database_name,sample_id,id,output,ksize,sequence_to_compare,database
2784,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k21-protein,C14_B003528_S62,C14_B003528_S62_ignore-abundance=True_tabula-muris-k21-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/C14_B003528_S62_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
2785,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k27-protein,C14_B003528_S62,C14_B003528_S62_ignore-abundance=True_tabula-muris-k27-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k27-protein/C14_B003528_S62_ignore-abundance=True.csv,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
2786,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k33-protein,C14_B003528_S62,C14_B003528_S62_ignore-abundance=True_tabula-muris-k33-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k33-protein/C14_B003528_S62_ignore-abundance=True.csv,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
2787,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k51-protein,C14_B003528_S62,C14_B003528_S62_ignore-abundance=True_tabula-muris-k51-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/
4368,True,s3://olgabot-maca/lung_cancer/sourmash_v4/D1_B003125_S25.signature,tabula-muris-k21-protein,D1_B003125_S25,D1_B003125_S25_ignore-abundance=True_tabula-muris-k21-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/D1_B003125_S25_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/


In [31]:
samples_no_sample_id = samples_subset.drop(columns=['sample_id'])
samples_no_sample_id = samples_no_sample_id.set_index('id')
print(samples_no_sample_id.shape)
samples_no_sample_id.head()

(168, 7)


,ignore_abundance,signature,database_name,output,ksize,sequence_to_compare,database
id,,,,,,,
C14_B003528_S62_ignore-abundance=True_tabula-muris-k21-protein,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k21-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/C14_B003528_S62_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/
C14_B003528_S62_ignore-abundance=True_tabula-muris-k27-protein,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k27-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k27-protein/C14_B003528_S62_ignore-abundance=True.csv,27,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k27-protein/tabula-muris-k27-protein/
C14_B003528_S62_ignore-abundance=True_tabula-muris-k33-protein,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k33-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k33-protein/C14_B003528_S62_ignore-abundance=True.csv,33,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k33-protein/tabula-muris-k33-protein/
C14_B003528_S62_ignore-abundance=True_tabula-muris-k51-protein,True,s3://olgabot-maca/lung_cancer/sourmash_v4/C14_B003528_S62.signature,tabula-muris-k51-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k51-protein/C14_B003528_S62_ignore-abundance=True.csv,51,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k51-protein/tabula-muris-k51-protein/
D1_B003125_S25_ignore-abundance=True_tabula-muris-k21-protein,True,s3://olgabot-maca/lung_cancer/sourmash_v4/D1_B003125_S25.signature,tabula-muris-k21-protein,s3://olgabot-maca/lung_cancer/sourmash_search/tabula-muris-k21-protein/D1_B003125_S25_ignore-abundance=True.csv,21,protein,s3://olgabot-maca/facs/sourmash_index_all/tabula-muris-k21-protein/tabula-muris-k21-protein/


In [32]:
# folder = f'../sourmash/lung_cancer_v4/search_{database_name}'
folder = f'../sourmash/lung_cancer_v4/search_protein_databases'
! mkdir $folder

mkdir: ../sourmash/lung_cancer_v4/search_protein_databases: File exists


In [33]:
import json

config = 	{
		"program": "../../../reflow/sourmash_search.rf",
		"runs_file": "samples.csv"
	}

samples_no_sample_id.to_csv(f'{folder}/samples.csv', index=True)


with open(f'{folder}/config.json', 'w') as f:
    json.dump(config, f)

In [34]:
pwd

'/Users/olgabot/code/kmer-hashing/notebooks'

In [35]:
ls -lha $folder

total 136
drwxr-xr-x  4 olgabot  staff   136B Sep 24 04:29 ./
drwxr-xr-x  6 olgabot  staff   204B Sep 21 10:31 ../
-rw-r--r--  1 olgabot  staff    77B Oct  5 17:51 config.json
-rw-r--r--  1 olgabot  staff    62K Oct  5 17:51 samples.csv


In [36]:
! wc -l $folder/samples.csv

     169 ../sourmash/lung_cancer_v4/search_protein_databases/samples.csv
